# Packages

In [32]:
import matplotlib.pyplot as plt
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import numpy as np
from rpy2.robjects import r, pandas2ri

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython


import sys
sys.path.append('/home/divar/projects/geometric-sampling')
import geometric_sampling

import geometric_sampling as gs
from geometric_sampling.search.astar import AStar
from geometric_sampling.design import Design
from geometric_sampling.criteria.var_nht import VarNHT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [33]:
%%R
#install.packages("sampling")
library(sampling)

In [34]:
%%R
Ppi <- function(Pi) {
  
  N <- length(Pi)
  #SOME ERROR MESSAGES
  if (N < 2) {
    rlang::abort("The sampling designs should be define on a set of more than one element. (length(Pi) > 1)")
  }
  
  for (k in 1:N) {
    if (Pi[k] >= 1 | Pi[k] <= 0) {
      rlang::abort("Pi is not a vector of probabilities (0 <= p < 1)")
    }
  }
  
  if (as.integer(round( sum(Pi) , 9)) - round( sum(Pi) , 9) != 0) {
    rlang::abort("The sum of the first order inclusion probabilities should be an integer")
  }
  
  
  s <- c()
  c <- c()
  kr <- c()
  alpha <-c()
  sum <-0
  r<-1
  r_prev<-0
  n_<-sum(Pi)
  
  for (k in 1:N) {
    prev_sum<-sum
    sum<-sum+Pi[k]
    if (sum>=r)
    {
      kr[r] <- k
      alpha[k] <-r-prev_sum
      
      int <- sqrt( (1 - Pi[k]) / (1 - alpha[k]) )
      s[k] <- round(int, 8)
      r_prev<-r
      r<-r+1
    }
    
    else {
      inter <- sqrt( Pi[k] / (r_prev + 1 - prev_sum) )
      s[k] <- round(inter, digits = 15)
    }
    
    c[k] <- sqrt(1 - s[k]^2)
    
  }
  
  # ce point n'est pas joli, mais je n'ai pas trouvé l'erreur. A retravailler..
  #print(kr)
  if(max(kr)!=length(Pi)){
    kr<-cbind(kr,length(Pi))
    r<-r+1
    r_prev<-r_prev+1}
  #print(kr)
  V <- matrix(0, nrow = N , ncol = r_prev)
  V[1, 1] = 1
  if ((r_prev-1) != 0) {
    for (r in 1:(r_prev-1)) {
      V[kr[r] + 1, r + 1] = 1
    }
  }
  
  for (k in 1:(N-1)) {
    L <- V[k, ]
    M <- V[k + 1,]
    V[k, ] <- s[k] * L - c[k] * M
    V[k + 1, ] <- c[k] * L + s[k] * M
  }
  return(V)
}




Drawing_Dsd <- function(v, s = 1, B = FALSE, seed = NULL){
  
  
  if (is.numeric(v)) {
    return(.dsd_sampling_mult(v, s, B, seed))
  }
  else{ return(.dsd_sampling_mult_complex(v, s, B, seed))}
}


.dsd_sampling_mult_complex <- function(v, s, B, seed){
  
  if (s == 1) {
    return(.dsd_sampling_01_B_C_complex(v, B, seed))
  }
  
  else{
    echant <- replicate(s, .dsd_sampling_01_B_C_complex(v, B, seed))
    colnames(echant) <- paste("Sample", 1:s)
    return(echant)
  }
}


.dsd_sampling_mult <- function(v = NULL, s, B,seed){
  
  if (s == 1) {
    return(.dsd_sampling_01_B_C(v, B, seed))
  }
  else{
    echant <- replicate(s, .dsd_sampling_01_B_C(v, B, seed))
    colnames(echant) <- paste("Sample", 1:s)
    return(echant)
  }
}

.dsd_sampling_01_B_C_complex <- function(v, B = TRUE, seed){
  
  indices <- .data <- NULL
  
  N <- nrow(v)
  n <- ncol(v)
  echant <- rep(0, N)
  
  if (!is.null(seed)) {
    set.seed(seed)
  }
  ref <- stats::runif(n)
  
  #Step 1: Sampling the first element
  w <- v
  
  total <- 0
  i <- 0
  pi1 <- Re( diag( v %*% t(Conj(v)) ) )
  
  if (length(pi1[pi1 < 0]) != 0 | length(pi1[pi1 >= 1]) != 0) {
    rlang::abort("The matrix v given as input doesn't suit to the input expected (See the functions pgd and periodic_dsd)")
  }
  
  while (total < ref[1]) {
    i <- i + 1
    total <- total + ( pi1[i] / n )
  }
  echant[i] <- 1
  
  M <- v[i,]
  e1 <- M / c(Re (sqrt (t(M) %*% Conj(M)) ) )
  
  
  #Step 2: Sampling the n-1 others elements
  for (j in 1:(n-1)) {
    
    r <- n-j
    inter <- v %*% Conj(e1)
    pi1 <- pi1 - t(inter * Conj(inter))
    pi2 <- Re( 1 / r*pi1 )
    
    
    total <- 0
    i <- 0
    
    while (total < ref[j+1]) {
      i <- i + 1
      total <- total + pi2[i]
    }
    echant[i] <- 1
    
    
    w <- w - t( t(Conj(e1)) %*% t(w) ) %*% t(e1)
    L <- w[i, ]
    e1 <- L / c(Re(sqrt (t(L) %*% Conj(L) )))
    
  }
  
  if(B) {
    return(echant)
  }
  else {
    return((1:N)[echant==1])
  }
  
}
.dsd_sampling_01_B_C <- function(v = NULL, B = TRUE, seed){
  
  indices <- .data <- NULL
  
  N <- nrow(v)
  n <- ncol(v)
  echant <- rep(0, N)
  
  if (!is.null(seed)) {
    set.seed(seed)
  }
  ref <- stats::runif(n)
  
  #First step: Sampling the first element
  w <- v
  
  
  total <- 0
  i <- 0
  pi1 <- diag(v  %*% t(v))
  
  while (total < ref[1]) {
    i <- i + 1
    total <- total + ( pi1[i] / n )
  }
  echant[i] <- 1
  
  
  l <- v[i,]
  e1 <- l / as.numeric( sqrt( t(l) %*% l ) )
  
  
  #Step 2: Sampling the n-1 others elements
  for (j in 1:(n-1)) {
    
    r <- n-j
    inter <- (v %*% e1)
    pi1 <- pi1 - t( inter * inter )
    pi2 <- 1 / r * pi1
    
    
    total <- 0
    i <- 0
    
    while (total < ref[j+1]) {
      i <- i + 1
      total <- total + pi2[i]
    }
    echant[i] <- 1
    
    
    
    w <- w - (w %*% e1) %*% t(e1)
    L <- w[i,]
    e1 <- L / as.numeric( sqrt( t(L) %*% L ))
    
  }
  if(B) {
    return(echant)
  }
  else {
    return((1:N)[echant==1])
  }
  
}

# Parameters

In [35]:
%%R
N <- 100
n = 10
set.seed(1983)
mu <- 100
b = 5
sigma_y <- 10
#target_rhos <- c(0.9, 0.8, 0.75)

m = 2000
balance_on_intercept = 0



In [36]:
%%R
library(sampling)

target_rhos <- c(0.65, 0.75, 0.85, 0.95)
# --- generate y only once ---
y <- rnorm(N, mu, sigma_y)

# Precompute all z's
z_list <- list()
for (rho_z in target_rhos) {
  sigma_zerr <- sqrt(var(y) * (1/rho_z^2 - 1))
  z_list[[as.character(rho_z)]] <- b * (y + rnorm(N, 0, sigma_zerr))
}

# Precompute all p's
p_list <- list()
for (rho_p in target_rhos) {
  sigma_perr <- sqrt(var(y) * (1/rho_p^2 - 1))
  p_list[[as.character(rho_p)]] <- b * (y + rnorm(N, 0, sigma_perr))
}

# You now have exactly 3 z and 3 p (not 9!)

In [37]:
%%R
vars_results <- list()
for (rho_z in target_rhos) {
  for (rho_p in target_rhos) {
    z <- z_list[[as.character(rho_z)]]
    p <- p_list[[as.character(rho_p)]]
    pik <- inclusionprobabilities(p, n)

    print(round(c(cor(y,z),cor(y,pik)),2))

    label <- paste0("cor_zy_", rho_z*10, "_py_", rho_p*10)
    data <- data.frame(y=y, z=z, pik=pik)
   
    pik = data$pik
    # Other designs variances
    pikl_sys = UPsystematicpi2(pik)
    pikl_max = UPmaxentropypi2(pik)
    pikl_mid = UPmidzunopi2(pik)
    pikl_til = UPtillepi2(pik)
    
    z_hat = data$z / pik
    y_hat = data$y / pik
    var__sys <- t(z_hat) %*% (pikl_sys - (pik %*% t(pik))) %*% z_hat
    var__max <- t(z_hat) %*% (pikl_max - (pik %*% t(pik))) %*% z_hat
    var__mid <- t(z_hat) %*% (pikl_mid - (pik %*% t(pik))) %*% z_hat
    var__til <- t(z_hat) %*% (pikl_til - (pik %*% t(pik))) %*% z_hat
    var__srs <- (N**2)*(1-n/N)*(1/n)*var(z)
    var__sys_y <- t(y_hat) %*% (pikl_sys - (pik %*% t(pik))) %*% y_hat
    var__max_y <- t(y_hat) %*% (pikl_max - (pik %*% t(pik))) %*% y_hat
    var__mid_y <- t(y_hat) %*% (pikl_mid - (pik %*% t(pik))) %*% y_hat
    var__til_y <- t(y_hat) %*% (pikl_til - (pik %*% t(pik))) %*% y_hat
    var__srs_y <- (N**2)*(1-n/N)*(1/n)*var(y)
    
    # Save all variables for this scenario in a list:
    vars_results[[label]] <- list(
      var__sys = var__sys,
      var__max = var__max,
      var__mid = var__mid,
      var__til = var__til,
      var__srs = var__srs,
      var__sys_y = var__sys_y,
      var__max_y = var__max_y,
      var__mid_y = var__mid_y,
      var__til_y = var__til_y,
      var__srs_y = var__srs_y
    )
  }
}

[1] 0.71 0.72
[1] 0.71 0.60
[1] 0.71 0.85
[1] 0.71 0.96
[1] 0.80 0.72
[1] 0.8 0.6
[1] 0.80 0.85
[1] 0.80 0.96
[1] 0.86 0.72
[1] 0.86 0.60
[1] 0.86 0.85
[1] 0.86 0.96
[1] 0.95 0.72
[1] 0.95 0.60
[1] 0.95 0.85
[1] 0.95 0.96


In [39]:
%%R
all_data = list()
for (rho_z in target_rhos) {
  for (rho_p in target_rhos) {
    print(c(rho_z,rho_p))
    z <- z_list[[as.character(rho_z)]]
    p <- p_list[[as.character(rho_p)]]

    pik <- inclusionprobabilities(p, n)

    label <- paste0("cor_zy_", rho_z*10, "_py_", rho_p*10)
    sort_index <- order(z / pik, decreasing = TRUE)
    y_ <- y[sort_index]; z_ <- z[sort_index]; pik_ <- pik[sort_index]; p_ = p[sort_index]
    N_ <- length(y_)
    dat <- data.frame(y=y_, z=z_, pik=pik_)
    all_data[[label]] <- dat
    Base <- Ppi(pik_)
    Ppi_mat <- Base %*% t(Base)
    Dpi_inv <- diag(1 / pik_)
    I_N <- diag(N_)
    A <- (I_N - Ppi_mat) * Ppi_mat
    var_ht_z <- as.numeric(t(z_) %*% Dpi_inv %*% A %*% Dpi_inv %*% z_)
    var_ht_y <- as.numeric(t(y_) %*% Dpi_inv %*% A %*% Dpi_inv %*% y_)
    
    ones = rep(1, N_)
    ht_estimates_dsd_y <- numeric(m)
    ht_estimates_dsd_z <- numeric(m)
    ht_estimates_cube_y <- numeric(m)
    ht_estimates_cube_z <- numeric(m)
    
    for (i in 1:m) {
      samp <- Drawing_Dsd(Base, s = n, B = TRUE)
      sel_idx <- which(samp[, 1] == 1)
      ht_estimates_dsd_y[i] <- sum(y_[sel_idx] / pik_[sel_idx])
      ht_estimates_dsd_z[i] <- sum(z_[sel_idx] / pik_[sel_idx])
      if (balance_on_intercept == 1) {
        XX <- cbind(pik_, ones, z_)
      } else {
        XX <- cbind(pik_, z_)
      }
      cube_sample <- samplecube(XX, pik_, comment = FALSE)
      sel_cube <- which(cube_sample == 1)
      ht_estimates_cube_y[i] <- sum(y_[sel_cube] / pik_[sel_cube])
      ht_estimates_cube_z[i] <- sum(z_[sel_cube] / pik_[sel_cube])
    }
    
    var_cube_y  <- var(ht_estimates_cube_y)
    var_cube_z  <- var(ht_estimates_cube_z)
    mean_cube_y <- mean(ht_estimates_cube_y)
    mean_cube_z <- mean(ht_estimates_cube_z)
    var_dsd_y   <- var_ht_y
    var_dsd_z   <- var_ht_z
    mean_dsd_y  <- mean(ht_estimates_dsd_y)
    mean_dsd_z  <- mean(ht_estimates_dsd_z)
    
    # --- Save extra ---
    save_df <- data.frame(
      var_ht_z = var_ht_z,
      var_ht_y = var_ht_y,
      var_cube_z = var_cube_z,
      var_cube_y = var_cube_y,
      mean_cube_y = mean_cube_y,
      mean_cube_z = mean_cube_z,
      mean_dsd_y = mean_dsd_y,
      mean_dsd_z = mean_dsd_z,
      n = n,
      N = N_
    )
    write.csv(save_df, paste0(label, '_extra.csv'), row.names = FALSE)
  }
}

# Save these: one .csv per scenario
for (label in names(all_data)) {
  write.csv(all_data[[label]], paste0(label, ".csv"), row.names=FALSE)
}

[1] 0.65 0.65
[1] 0.65 0.75
[1] 0.65 0.85
[1] 0.65 0.95
[1] 0.75 0.65
[1] 0.75 0.75
[1] 0.75 0.85
[1] 0.75 0.95
[1] 0.85 0.65
[1] 0.85 0.75
[1] 0.85 0.85
[1] 0.85 0.95
[1] 0.95 0.65
[1] 0.95 0.75
[1] 0.95 0.85
[1] 0.95 0.95


In [40]:
bardia_balance_method = 'origin'
num_new_nodes          = 30
num_changes_lower      = 1
num_changes_upper      = 3
max_open_set_size      = 200000
switch_lower           = .7
switch_upper           = .9
max_iterations         = 5000
num_initial_nodes      = 3000
initial_design_to_use  = 100
num_top_restart_nodes  = 10
stuck_fraction         = 0.99
swap_iterations        = int(np.round(.7 * num_initial_nodes))  # ensure integer!
swap_distance          = 3
swap_units             = int(20)

show_results           = 1
random_restart_period  = 10000   # how often to inject random designs
random_injection_count = 500     # how many random designs to inject
prune_fraction         = .9
var_percent_exected    = .1  # how much of the variance to expect in the best design



In [41]:
import csv

filename = f"config_{bardia_balance_method}.csv"

config = {
    'bardia_balance_method': bardia_balance_method,
    'num_new_nodes': num_new_nodes,
    'num_changes_lower': num_changes_lower,
    'num_changes_upper': num_changes_upper,
    'max_open_set_size': max_open_set_size,
    'switch_lower': switch_lower,
    'switch_upper': switch_upper,
    'max_iterations': max_iterations,
    'num_initial_nodes': num_initial_nodes,
    'initial_design_to_use': initial_design_to_use,
    'num_top_restart_nodes': num_top_restart_nodes,
    'stuck_fraction': stuck_fraction,
    'swap_iterations': swap_iterations,
    'swap_distance': swap_distance,
    'swap_units': swap_units,
    'show_results': show_results,
    'random_restart_period': random_restart_period,
    'random_injection_count': random_injection_count,
    'prune_fraction': prune_fraction,
    'var_percent_exected': var_percent_exected
}

with open(filename, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=list(config.keys()))
    writer.writeheader()
    writer.writerow(config)